# Random Dev For Optimizer and It's Tests

In [1]:
from numpy.testing.utils import assert_equal

from brian2 import *
import numpy as np
from brian2tools import NevergradOptimizer, SkoptOptimizer

from nevergrad.optimization import optimizerlib
from nevergrad import instrumentation as inst

WARNING    /home/alteska/anaconda3/envs/brian_opt/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
 [py.warnings]


In [2]:
candidates, parameters = [], []

# create input and output
input_traces = np.zeros((10,5))*volt
for i in range(5):
    input_traces[5:,i]=i*10*mV

output_traces = 10*nS*input_traces

model = Equations('''
    I = g*(v-E) : amp
    g : siemens (constant)
    E : volt (constant)
    ''')

# setup the nevergrad optimizer
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000)

for _ in range(10):
    cand = optim.ask()
    candidates.append(cand)
    parameters.append(list(cand.args))

In [3]:
candidates[0].args

(2.0696796857016686, 4.111273868640625)

In [4]:
x = [1,2,3]
y = [v * 5 for v in x ]
y

[5, 10, 15]

In [5]:
y = [list(v.args) for v in candidates]
y == parameters

True

In [6]:
np.random.random((10,2))

array([[0.82270557, 0.3225719 ],
       [0.60545623, 0.570278  ],
       [0.28894298, 0.17716415],
       [0.26804336, 0.50077974],
       [0.68290855, 0.54256322],
       [0.63391918, 0.32059107],
       [0.23189809, 0.04060585],
       [0.44165974, 0.87246838],
       [0.22103481, 0.28122694],
       [0.00144071, 0.53549554]])

In [7]:
n_opt = NevergradOptimizer(bounds=[[0,1]], parameter_names=['a'])
param = n_opt.ask(10)
assert_equal(np.shape(param),(10,1))

print(np.shape(param),(10,1))


Candidate(args=(0.19543895540744194,), kwargs={})
Candidate(args=(0.4474251539315146,), kwargs={})
Candidate(args=(0.24798151248045908,), kwargs={})
Candidate(args=(0.5691340977826351,), kwargs={})
Candidate(args=(0.6959812277543949,), kwargs={})
Candidate(args=(0.4250549796472558,), kwargs={})
Candidate(args=(0.39644830882820636,), kwargs={})
Candidate(args=(0.20622516982248618,), kwargs={})
Candidate(args=(0.34541958623738556,), kwargs={})
Candidate(args=(0.7969281553422587,), kwargs={})
(10, 1) (10, 1)


In [8]:
param2 = param
param2

[[0.19543895540744194],
 [0.4474251539315146],
 [0.24798151248045908],
 [0.5691340977826351],
 [0.6959812277543949],
 [0.4250549796472558],
 [0.39644830882820636],
 [0.20622516982248618],
 [0.34541958623738556],
 [0.7969281553422587]]

In [9]:
np.testing.assert_array_less(param2, 1)

In [10]:
np.where(param == param2)

(array([0]),)

In [11]:
np.shape([[1,2]])

(1, 2)

In [12]:
np.shape([[1,2], [1,2]])

(2, 2)

In [13]:
error = np.random.random((10))
error

array([0.4736548 , 0.2407261 , 0.81880626, 0.12532975, 0.00320145,
       0.77977349, 0.66657014, 0.84442709, 0.42708088, 0.88335053])

In [14]:
n_opt = NevergradOptimizer(bounds=[[0, 1], [0, 1]], parameter_names=['a', 'b'])
param = n_opt.ask(10)
error = np.random.random((10))

n_opt.tell(param, error)
ans = n_opt.recommend()

Candidate(args=(0.32537889727571406, 0.33410678973101593), kwargs={})
Candidate(args=(0.1506574441799614, 0.36115201250108997), kwargs={})
Candidate(args=(0.6993648402934443, 0.3174487583014535), kwargs={})
Candidate(args=(0.24649012453833918, 0.7697255502276387), kwargs={})
Candidate(args=(0.4159722785492307, 0.3021424796114356), kwargs={})
Candidate(args=(0.6723722392725989, 0.4123064179899561), kwargs={})
Candidate(args=(0.6078968801600019, 0.6030595114003007), kwargs={})
Candidate(args=(0.7865619133585569, 0.47333453950279647), kwargs={})
Candidate(args=(0.7222774860882015, 0.25005262104273573), kwargs={})
Candidate(args=(0.71290841124451, 0.7789482276824866), kwargs={})


In [15]:
x = np.array((ans.args))
y = np.array(param)
np.where(y == x)

(array([1, 1]), array([0, 1]))

## Dev Optimizer Test

In [16]:
import numpy as np

from random import choice
from string import digits, ascii_lowercase
from numpy.testing.utils import assert_equal, assert_raises
from brian2tools import Optimizer, NevergradOptimizer, SkoptOptimizer

```
@pytest.fixture
def item():
    return Item()

def test_item_created(item):
    # ensure item was created successfully
    assert item.value == 'something expected'

def test_edit_item(item):
    # ensure item is edited successfully
    item.edit()
```

In [17]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

In [18]:
for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    n_samples = np.random.randint(1, 30)
    errors = np.random.rand(n_samples)
    print(errors)
    

    n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
    param = n_opt.ask(n_samples)

    assert_equal(np.shape(param), (n_samples, n))

    for i in np.arange(1, n):
        assert all(np.array(param)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(param)[:,i-1] >  0), 'Values in params are smaller than required'

[0.57324091 0.11050388 0.94318175 0.54519761 0.02463069 0.23890581
 0.85301153 0.70948191 0.72505201 0.50965362 0.96358027 0.82553064
 0.85396862 0.42944482 0.79032437 0.27022979 0.99788609 0.44491896
 0.93317951 0.48164485 0.93049566 0.99832142 0.08204166 0.01020527
 0.73667082]
Candidate(args=(0.7733390053611697,), kwargs={})
Candidate(args=(0.8672052223119333,), kwargs={})
Candidate(args=(0.456149473779344,), kwargs={})
Candidate(args=(0.3169201174752102,), kwargs={})
Candidate(args=(0.20213142274200657,), kwargs={})
Candidate(args=(0.7484825324111821,), kwargs={})
Candidate(args=(0.3922050684387144,), kwargs={})
Candidate(args=(0.9031449561426348,), kwargs={})
Candidate(args=(0.7646396758764908,), kwargs={})
Candidate(args=(0.5789768762895582,), kwargs={})
Candidate(args=(0.3174814143745123,), kwargs={})
Candidate(args=(0.45681138679293987,), kwargs={})
Candidate(args=(0.8352509380218389,), kwargs={})
Candidate(args=(0.30411635376132873,), kwargs={})
Candidate(args=(0.6073811210539

In [19]:
xx = np.zeros((10,2))
xx[:, 1] = np.arange(1,11,1.)
xx

array([[ 0.,  1.],
       [ 0.,  2.],
       [ 0.,  3.],
       [ 0.,  4.],
       [ 0.,  5.],
       [ 0.,  6.],
       [ 0.,  7.],
       [ 0.,  8.],
       [ 0.,  9.],
       [ 0., 10.]])

In [20]:
# Prep data to validate init on
chars = digits + ascii_lowercase
L = ["".join([choice(chars) for i in range(2)]) for j in range(10)]
L

a = np.arange(1, 11, 1.).reshape(10, 1)
b = np.zeros(10).reshape(10, 1)
lx = np.hstack((b, a)).tolist()
lx

[[0.0, 1.0],
 [0.0, 2.0],
 [0.0, 3.0],
 [0.0, 4.0],
 [0.0, 5.0],
 [0.0, 6.0],
 [0.0, 7.0],
 [0.0, 8.0],
 [0.0, 9.0],
 [0.0, 10.0]]

In [21]:
for n in np.arange(1,11):
    par_names, bounds = L[:n], lx[:n]
    n_samples = numpy.random.randint(1, 30)
    
    n_opt = NevergradOptimizer(bounds=bounds, parameter_names=par_names)
    param = n_opt.ask(n_samples)
    
    assert_equal(np.shape(param), (n_samples,n))
    
    for i in np.arange(1, n):
        assert all(np.array(param)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(param)[:,i-1] >  0), 'Values in params are bigger than required'


Candidate(args=(0.663943762065647,), kwargs={})
Candidate(args=(0.5857469328894169,), kwargs={})
Candidate(args=(0.679683328105813,), kwargs={})
Candidate(args=(0.6411583771034208,), kwargs={})
Candidate(args=(0.33328342148354234,), kwargs={})
Candidate(args=(0.6815264652843694,), kwargs={})
Candidate(args=(0.8733022920808253,), kwargs={})
Candidate(args=(0.7385217893490927, 1.5051123087853955), kwargs={})
Candidate(args=(0.7519724145167579, 0.8393418603281966), kwargs={})
Candidate(args=(0.7270406350633174, 1.2484277485193722), kwargs={})
Candidate(args=(0.3608889022265882, 1.0370341751678676, 1.399897663256846), kwargs={})
Candidate(args=(0.45635186265313393, 1.6320251472246174, 2.557024523876095), kwargs={})
Candidate(args=(0.5318018201953769, 1.504795732479723, 0.5018579493389022), kwargs={})
Candidate(args=(0.2580397723660568, 1.1137977109777548, 1.971166930193397), kwargs={})
Candidate(args=(0.36926533062747563, 1.6516717240793775, 1.1628241753398432), kwargs={})
Candidate(args=(

In [22]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    n_samples = np.random.randint(1, 30)

    n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
    params = n_opt.ask(n_samples)

    assert_equal(np.shape(params), (n_samples, n))

    for i in np.arange(1, n):
        assert all(np.array(params)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(params)[:,i-1] >  0), 'Values in params are smaller than required'

    errors = np.random.rand(n_samples)
    n_opt.tell(params, errors)

    print(n_opt.recommend())

Candidate(args=(0.6301749194275708,), kwargs={})
Candidate(args=(0.30888153948118896,), kwargs={})
Candidate(args=(0.3681396264765404,), kwargs={})
Candidate(args=(0.4868763070536418,), kwargs={})
Candidate(args=(0.6212429234103274,), kwargs={})
Candidate(args=(0.40669060278577907,), kwargs={})
Candidate(args=(0.845615773188283,), kwargs={})
Candidate(args=(0.5891816584852841,), kwargs={})
Candidate(args=(0.8192804593357816,), kwargs={})
Candidate(args=(0.28208797974015154,), kwargs={})
Candidate(args=(0.5255552302747822,), kwargs={})
Candidate(args=(0.7022993632240399,), kwargs={})
Candidate(args=(0.2734306146588793,), kwargs={})
Candidate(args=(0.845615773188283,), kwargs={})
Candidate(args=(0.680741032120679, 0.4798144624365184), kwargs={})
Candidate(args=(0.680741032120679, 0.4798144624365184), kwargs={})
Candidate(args=(0.772412386584842, 0.8101354249271421, 1.5305196377965309), kwargs={})
Candidate(args=(0.14556206219104695, 0.7066447542357488, 1.1602838098685118), kwargs={})
Can

In [23]:
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

n=1
par_names, bound = labels[:n], bounds[:n]
n_samples = np.random.randint(1, 30)

n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
params = n_opt.ask(n_samples)

assert_equal(np.shape(params), (n_samples, n))

for i in np.arange(1, n):
    assert all(np.array(params)[:,i-1] <= i), 'Values in params are bigger than required'
    assert all(np.array(params)[:,i-1] >  0), 'Values in params are smaller than required'

errors = np.random.rand(n_samples)
n_opt.tell(params, errors)

ans = n_opt.recommend()


Candidate(args=(0.2544193126820437,), kwargs={})
Candidate(args=(0.6054397155613874,), kwargs={})
Candidate(args=(0.3493736620770882,), kwargs={})


In [24]:
params

[[0.2544193126820437], [0.6054397155613874], [0.3493736620770882]]

In [25]:
n_opt.optim.create_candidate.from_data(params[0])

Candidate(args=(0.5793017107045426,), kwargs={}, data=[0.25441931])

In [26]:
params[0]

[0.2544193126820437]

In [27]:
n_opt.candidates[0].data

array([-0.97261114])

In [28]:
params

[[0.2544193126820437], [0.6054397155613874], [0.3493736620770882]]

In [29]:
er_min = (errors).argmin()
assert params[er_min] == list(ans.args), "Optimizer didn't return the minimal value"

In [30]:
from nevergrad.optimization.base import Candidate
Candidate(params[0], {}, [])

Candidate(args=[0.2544193126820437], kwargs={}, data=[])

### Write Separate Tell Test 

In [31]:
n_samples = np.random.randint(1, 30)
params = np.random.rand(n_samples, 2)
data = np.random.rand(n_samples, 2)
errors = np.random.rand(n_samples)

candidates = []
for i, p in enumerate(params):
#     candidates.append(Candidate(p, {}, data[i]))
    candidates.append(Candidate(p, {}, []))

print(candidates)

[Candidate(args=[0.4868167  0.38079916], kwargs={}, data=[]), Candidate(args=[0.58258026 0.45163728], kwargs={}, data=[]), Candidate(args=[0.29457083 0.93177098], kwargs={}, data=[]), Candidate(args=[0.35403316 0.29803552], kwargs={}, data=[]), Candidate(args=[0.58132172 0.85639452], kwargs={}, data=[]), Candidate(args=[0.0172365  0.28141244], kwargs={}, data=[]), Candidate(args=[0.09225434 0.66257657], kwargs={}, data=[]), Candidate(args=[0.60088648 0.19015773], kwargs={}, data=[]), Candidate(args=[0.38953431 0.00222286], kwargs={}, data=[]), Candidate(args=[0.4108685 0.5596885], kwargs={}, data=[]), Candidate(args=[0.09711831 0.10090259], kwargs={}, data=[]), Candidate(args=[0.50987642 0.56071553], kwargs={}, data=[]), Candidate(args=[0.91699254 0.63287313], kwargs={}, data=[]), Candidate(args=[0.93210222 0.22097409], kwargs={}, data=[]), Candidate(args=[0.31330263 0.30450056], kwargs={}, data=[]), Candidate(args=[0.91106384 0.53359027], kwargs={}, data=[]), Candidate(args=[0.0509603

AssertionError: Expected 1 values but got 0

In [33]:
n_opt = NevergradOptimizer(bounds=[[0,1]], parameter_names=['a'])

In [35]:
n_opt.candidates = candidates

In [36]:
n_opt.tell(params, errors)

In [37]:
n_opt.recommend()

AssertionError: Expected 1 values but got 0

AssertionError: Expected 1 values but got 0

In [ ]:
cand = [list(v.args) for v in candidates]
if np.all(params == cand):
    print('ok')

if np.all(params == [list(v.args) for v in candidates]):
    print('ok')

In [ ]:
from nevergrad.optimization.base import Candidate

labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    n_samples = np.random.randint(1, 30)
    params = np.random.rand(n_samples, 2)
    errors = np.random.rand(n_samples)

    
    n_opt = NevergradOptimizer(bounds=bound, parameter_names=par_names)
    
    assert_equal(np.shape(params), (n_samples, n))

    for i in np.arange(1, n):
        assert all(np.array(params)[:,i-1] <= i), 'Values in params are bigger than required'
        assert all(np.array(params)[:,i-1] >  0), 'Values in params are smaller than required'

    n_opt.tell(params, errors)

    ans = n_opt.recommend()
    er_min = (errors).argmin()
    assert params[er_min] == list(ans.args), "Optimizer didn't return the minimal value"



In [50]:
sk_opt = []
labels = ["".join([choice(digits + ascii_lowercase) for i in range(2)]) for j in range(10)]

bounds = np.zeros((10,2))
bounds[:, 1] = np.arange(1,11,1.)

for n in np.arange(1,11):
    par_names, bound = labels[:n], bounds[:n]
    sk_opt.append(SkoptOptimizer(bounds=bound, parameter_names=par_names))

In [56]:
skopt_optimizer = sk_opt
for n, skopt in enumerate(skopt_optimizer):
    print(n, skopt.optim.dimensions)

AttributeError: 'Optimizer' object has no attribute 'dimensions'